In [2]:
# python 3.7
# -*- coding: utf-8 -*-

#!/usr/bin/env python
# coding: utf-8
"""
1. using all publications of researchers with different weight as input to generate user profiles
2. pretrain word2vec model window_5.model.bin and candidate_paper.csv are available via google drive link,
you can download the files and
change the path in this script so as to run the script successfully.
3. result saved in rank_result_all_weight/weight_CP.csv
"""
import sys
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
import pandas as pd
from datetime import datetime

# define DocSim class to calculate document similarities
class DocSim(object):
    def __init__(self, w2v_model , stopwords=[]):
        self.w2v_model = w2v_model
        self.stopwords = stopwords

    def vectorize(self, doc):
        """Identify the vector values for each word in the given document"""
        doc = str(doc)
        doc = doc.lower()
        words = [w for w in doc.split(" ") if w not in self.stopwords]
        word_vecs = []
        for word in words:
            try:
                vec = self.w2v_model[word]
                word_vecs.append(vec)
            except KeyError:
                # Ignore, if the word doesn't exist in the vocabulary
                pass

        # Assuming that document vector is the mean of all the word vectors
        vector = np.mean(word_vecs, axis=0)
        return vector

In [3]:
# load pre-train model on my own corpus
model = 'window_5.model.bin'
w2v_model = KeyedVectors.load_word2vec_format(model, binary=True)
ds = DocSim(w2v_model)

### convert a text file to w2v vector and save the vector to a new file

In [4]:
import pandas as pd

def get_file_name(file_path):
    # input a cadidate paper file path, return the paperID as file name
    # file_path = "/AAMAS/AAMAS2005/p100-pechoucek.pdf\n"
    # return "AAMAS05-p100-pechoucek"
    paperName = file_path.split("/")[-1].replace(".pdf\n", "")
    confName = file_path.split("/")[-3]
    year = file_path.split("/")[-2][-2:]
    name = f"{confName}{year}-{paperName}"
    return name

def userfile2vec(file_path, name):
    with open(file_path, "r") as f:
        text = f.readline()
    vec = ds.vectorize(text)
    # save the vector to a local file
    np.savetxt(f"{name}.csv",vec, delimiter=',' )
    # load vector
    vec = np.loadtxt(f"{name}.csv",delimiter=',')
    return vec

def paper2vec(paperID, text):
    name = get_file_name(paperID)
    vec = ds.vectorize(text)
    np.savetxt(f"candidate_vectors/{name}.csv",vec, delimiter=',' )
    # load vector
    vec = np.loadtxt(f"candidate_vectors/{name}.csv",delimiter=',')
    return vec

def iter_files(rootDir):
    """Traverse the root directory to return a list of txt file paths for all leaf nodes"""
    file_name_list = []
    for root,dirs,files in os.walk(rootDir):
        for file in files:
            file_name = os.path.join(root,file)
            file_name_list.append(file_name)
    return file_name_list

In [6]:
# read all candidate papers info, contain two columns: paper ID and paper content

candidate_papers = pd.read_csv('candidate_papers.csv')
# rename columns in user_profile and candidate_papers
candidate_papers.columns = ['paperID', 'paperText']
fail_ls = []
for i in range(len(candidate_papers['paperID'])):
    paperID = candidate_papers['paperID'][i]
    text = candidate_papers['paperText'][i]
    try:
        vec = paper2vec(paperID, text)
    except:
        fail_ls.append(paperID)
pd.DataFrame(fail_ls).to_csv("fail_candidate_vectors.csv")

/Users/sherry/Library/Python/3.9/lib/python/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/sherry/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
